## Deliverable 2. Create a Customer Travel Destinations Map.

In [45]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [46]:
# 1. Import the WeatherPy_database.csv file. 
database_path = os.path.join('..','Weather_Database','WeatherPy_Database.csv')
city_data_df = pd.read_csv(database_path)

In [47]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Enter the minimum temperature desired: '))
max_temp = float(input('Enter the maximum temperature desired: '))

In [48]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Los Llanos De Aridane,ES,28.6585,-17.9182,66.2,72,75,16.11,broken clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,51.8,62,20,26.46,few clouds
2,2,Marin,US,38.0666,-122.7344,50.0,93,90,4.65,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,48.0,80,50,8.01,scattered clouds
7,7,Mar Del Plata,AR,-38.0023,-57.5575,64.4,55,90,23.02,overcast clouds


In [49]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0         0
City               0
Country            2
Lat                0
Long               0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [50]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [51]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Long"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Long,Hotel Name
0,Los Llanos De Aridane,ES,66.20,broken clouds,28.6585,-17.9182,
1,Ushuaia,AR,51.80,few clouds,-54.8000,-68.3000,
2,Marin,US,50.00,overcast clouds,38.0666,-122.7344,
4,Mataura,NZ,48.00,scattered clouds,-46.1927,168.8643,
7,Mar Del Plata,AR,64.40,overcast clouds,-38.0023,-57.5575,
10,Penzance,GB,50.00,overcast clouds,50.1186,-5.5371,
12,Portland,US,45.00,overcast clouds,45.5234,-122.6762,
13,Dehloran,IR,52.97,clear sky,32.6941,47.2679,
15,Severo-Kurilsk,RU,23.54,snow,50.6789,156.1250,
16,Merritt Island,US,71.60,broken clouds,28.5392,-80.6720,


In [52]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    long = row['Long']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f'{lat},{long}'
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except:
        continue

In [53]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [54]:
# 8a. Create the output File (CSV)
output_data_file = './WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Weather}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp} F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Long"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure(center=(30,31),zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))